In [ ]:
from IPython.display import display, HTML
from datetime import datetime, timedelta
import imaplib
import email
import os
import zipfile
from google.colab import files

# Configurações do email
username = "gestaodecombustivel@rovemaenergia.com.br"
password = "clja pvdk mcnu cxzd"  # Altere para a sua senha de aplicativo
imap_server = "imap.gmail.com"

# Função para baixar PDFs e XMLs a partir da data especificada
def download_files_by_date(username, password, selected_date):
    try:
        # Conectar ao servidor IMAP
        mail = imaplib.IMAP4_SSL(imap_server)
        mail.login(username, password)

        # Selecionar a caixa de entrada
        mail.select("inbox")

        # Definir as datas para a busca
        since_date_str = selected_date.strftime("%d-%b-%Y")
        before_date_str = (selected_date + timedelta(days=1)).strftime("%d-%b-%Y")  # Um dia após a data selecionada

        # Buscar e-mails exatamente na data selecionada
        result, data = mail.search(None, f'SINCE {since_date_str} BEFORE {before_date_str}')

        # Listas para armazenar os arquivos baixados
        downloaded_boleto = []
        downloaded_nota_fiscal = []
        downloaded_xml = []

        # Verificar se a busca retornou e-mails
        if result == 'OK':
            email_ids = data[0].split()
            for email_id in email_ids:
                result, msg_data = mail.fetch(email_id, '(RFC822)')
                raw_email = msg_data[0][1]

                # Analisar o e-mail
                msg = email.message_from_bytes(raw_email)

                # Verificar se o e-mail tem múltiplas partes
                if msg.is_multipart():
                    for part in msg.walk():
                        # Verificar se a parte é PDF ou XML
                        content_type = part.get_content_type()
                        if content_type in ['application/pdf', 'application/xml']:
                            filename = part.get_filename()
                            if filename:
                                # Determinar se é um boleto, nota fiscal ou XML
                                if content_type == 'application/pdf':
                                    if 'boleto' in filename.lower():
                                        folder = "Boletos"
                                        downloaded_boleto.append(filename)
                                    elif 'nota fiscal' in filename.lower() or 'nf' in filename.lower():
                                        folder = "Notas Fiscais"
                                        downloaded_nota_fiscal.append(filename)
                                    else:
                                        folder = "Outros PDFs"
                                elif content_type == 'application/xml':
                                    folder = "XMLs"
                                    downloaded_xml.append(filename)

                                # Criar a pasta se não existir
                                if not os.path.exists(folder):
                                    os.makedirs(folder)

                                # Salvar o arquivo com o mesmo nome na pasta correspondente
                                filepath = os.path.join(folder, filename)
                                with open(filepath, 'wb') as f:
                                    f.write(part.get_payload(decode=True))
                                    print(f"Baixado: {folder}/{filename}")  # Exibe o arquivo baixado

        # Desconectar do servidor IMAP
        mail.logout()

    except Exception as e:
        print(f"Erro ao tentar se conectar ao servidor IMAP: {e}")

    return downloaded_boleto, downloaded_nota_fiscal, downloaded_xml

# Função para compactar arquivos em um ZIP
def zip_files(folder_name, files_list):
    with zipfile.ZipFile(f"{folder_name}.zip", 'w') as zipf:
        for file in files_list:
            zipf.write(os.path.join(folder_name, file), arcname=file)
    print(f"Arquivo {folder_name}.zip criado com sucesso!")

# Função a ser chamada a partir do HTML
def download_files(date):
    # Converte a data recebida do formato string para datetime
    selected_date = datetime.strptime(date, "%Y-%m-%d")
    downloaded_boleto, downloaded_nota_fiscal, downloaded_xml = download_files_by_date(username, password, selected_date)

    # Compactar e baixar boletos
    if downloaded_boleto:
        zip_files("Boletos", downloaded_boleto)
        files.download("Boletos.zip")

    # Compactar e baixar notas fiscais
    if downloaded_nota_fiscal:
        zip_files("Notas Fiscais", downloaded_nota_fiscal)
        files.download("Notas Fiscais.zip")

    # Compactar e baixar XMLs
    if downloaded_xml:
        zip_files("XMLs", downloaded_xml)
        files.download("XMLs.zip")

    if not downloaded_boleto and not downloaded_nota_fiscal and not downloaded_xml:
        print("Nenhum arquivo foi baixado.")

    # Reexibir a interface
    display(HTML(html_code))

# Registre a função no ambiente Colab
import google.colab
google.colab.output.register_callback('notebook.download_files', download_files)

# HTML para a interface
html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Download de PDFs e XMLs</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f4;
            margin: 0;
            padding: 20px;
        }
        h1 {
            color: #333;
        }
        form {
            background-color: #fff;
            padding: 20px;
            border-radius: 5px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            max-width: 400px;
            margin: auto;
        }
        label {
            display: block;
            margin-bottom: 8px;
            color: #555;
        }
        input[type="date"] {
            width: calc(100% - 22px);
            padding: 10px;
            margin-bottom: 15px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        input[type="submit"] {
            background-color: #28a745;
            color: white;
            padding: 10px 15px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            transition: background-color 0.3s ease;
            width: 100%;
        }
        input[type="submit"]:hover {
            background-color: #218838;
        }
        #result {
            margin-top: 20px;
            text-align: center;
        }
        .header-title {
            text-align: center;
            font-size: 24px;
            color: #333;
            margin-bottom: 20px;
        }
    </style>
</head>
<body>
    <h1 class="header-title">Gestão Combustível</h1>
    <h1>Download de PDFs e XMLs</h1>
    <form id="fileForm">
        <label for="date">Selecione a data:</label>
        <input type="date" id="date" name="date" required>
        <input type="submit" value="Download Arquivos">
    </form>

    <div id="result"></div>

    <script>
        document.getElementById('fileForm').onsubmit = function(e) {
            e.preventDefault();
            const date = document.getElementById('date').value;
            // Fazer uma chamada ao Colab aqui para o Python
            google.colab.kernel.invokeFunction('notebook.download_files', [date], {});
        };
    </script>
</body>
</html>
"""

# Exibir a interface
display(HTML(html_code))


Nenhum arquivo foi baixado.


Baixado: XMLs/NF-e_12241234274233025431550000000425981816301364.xml
Baixado: Notas Fiscais/DANFE_NFe000042598.PDF
Baixado: Boletos/BOLETO_NFe000042598.PDF
Arquivo Boletos.zip criado com sucesso!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Arquivo Notas Fiscais.zip criado com sucesso!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Arquivo XMLs.zip criado com sucesso!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Instalar as bibliotecas necessárias
!pip install pdfplumber pandas gspread oauth2client

# Carregar o PDF da nota fiscal
from google.colab import files
uploaded = files.upload()
pdf_file = next(iter(uploaded.keys()))  # Nome do arquivo carregado

# Importar bibliotecas necessárias
import pdfplumber
import pandas as pd
from gspread import service_account

# Função para extrair dados da nota fiscal
def extrair_dados_nota(pdf_file):
    dados = []
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            linhas = text.split('\n')
            dado1 = dado2 = dado3 = dado4 = None

            # Ajuste as condições abaixo de acordo com os seus dados
            for linha in linhas:
                if 'Quantidade' in linha:  # Substitua pelo critério de identificação do campo no PDF
                    dado1 = linha.split(':')[1].strip()  # Ajuste conforme necessário
                if 'Data de Emissão' in linha:  # Critério de busca para Data
                    dado2 = linha.split(':')[1].strip()
                if 'Valor Total' in linha:  # Critério de busca para Valor Total
                    dado3 = linha.split(':')[1].strip()
                if 'Valor Unitário' in linha:  # Critério de busca para Valor Unitário
                    dado4 = linha.split(':')[1].strip()

            # Certifique-se de que todos os dados foram encontrados antes de adicioná-los à lista
            if dado1 and dado2 and dado3 and dado4:
                dados.append([dado1, dado2, dado3, dado4])
    return dados

# Extrair os dados da nota fiscal
dados_nota = extrair_dados_nota(pdf_file)

# Autenticação para acessar o Google Sheets
gc = service_account(filename='from google.colab import files'
uploaded = files.upload()  # Isso abrirá um seletor de arquivos')  # Verifique o nome do arquivo JSON

# Criar um DataFrame com os dados
df = pd.DataFrame(dados_nota, columns=['QUANTIDADE', 'DATA EMISSAO', 'VALOR TOTAL DOS PRODUTOS', 'V.UNIT.'])

# Abrir sua planilha
sh = gc.open('Controle de Notas 2024')  # Certifique-se de que o nome da planilha está correto

# Selecionar a guia/aba onde você quer que os dados sejam colocados
worksheet = sh.worksheet('TESTE')  # Substitua 'TESTE' pelo nome correto da aba

# Adicionar os dados ao Google Sheets na aba selecionada
worksheet.update([df.columns.values.tolist()] + df.values.tolist())


SyntaxError: '(' was never closed (<ipython-input-4-25290f1bc61a>, line 43)

In [ ]:
from google.colab import files
uploaded = files.upload()  # Isso abrirá um seletor de arquivos


Saving inversores-pvh-ii-cb4b7af179fe.json to inversores-pvh-ii-cb4b7af179fe.json
